<a href="https://colab.research.google.com/github/akamojo/PAD-project/blob/master/part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PAD 2nd Assignment PART 1
- Mitchell Galvao MIEI 41646
- Urszula Walińska MIEI 56556

## Implementation of  the  multi‐word  Relevant  Expressions  LocalMaxs  extractor

In [0]:
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

     |████████████████████████████████| 92kB 764kB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-o15yijb4/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
import re
from tqdm import tqdm_notebook as tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Loading 2 mln corpus

In [0]:
file = open("/content/drive/My Drive/STUDIA/SEM 8/pad/en1.8m.txt", "r", encoding="utf8")
#file = open("en1.8m.txt", "r", encoding="utf8")

whole_text = file.read()
text = whole_text.split('\n')

## Data pre-processing  
Separation from words the characters that do not change the text semantics.

In [0]:
def append_space(text, pattern, left=True):
    p = re.compile("[" + pattern + "]")
    
    for i, m in enumerate(p.finditer(text)):
        if left:
            text = text[: m.start() + i] + " " + text[m.start() + i :]
        else:
            text = text[: m.start() + 1 + i] + " " + text[m.start() + 1 + i :]
            
    return text

In [0]:
def preprocessing(text):
    for i in range(len(text)):
        text[i] = append_space(text[i], ",\])>")
        text[i] = append_space(text[i], ".")
        text[i] = append_space(text[i], "\[(<", False)
      
    return text

## Function for building n-grams  
Based on the list of words.

In [0]:
def build_n_gram(text_arr, n_gram, grams):
    cur_gram = []
    count_gram = 0

    for i in range(len(text_arr)):

        cur_gram = [text_arr[i]]

        for j in range(i + 1, i + n_gram):
            if j < len(text_arr):
                cur_gram.append(text_arr[j])

        if len(cur_gram) == n_gram:
            try:
                grams[" ".join(cur_gram)] += 1
            except KeyError:
                grams[" ".join(cur_gram)] = 1

    return grams

## Building dictionary  
Which consists of n-grams and their frequency.

In [0]:
def build_dict(text, n_s):
    dictionary = []
    for n in range(n_s):
        dictionary.append(dict())
    
    for n in range(n_s):
        for line in text:
            dictionary[n] = build_n_gram(line.split(), n + 1, dictionary[n])
        
    return dictionary

## SCP_F measure

In [0]:
def SCP_f(words, dictionary):
    words_arr = words.split()
    
    numerator = dictionary[len(words_arr) - 1][words] ** 2
    F = 1 / (len(words_arr) - 1)
    denominator = 0
    
    for i in range(len(words_arr) - 1):
        left = words_arr[0:i+1]
        right = words_arr[i+1:]
        
        denominator += dictionary[len(left) - 1][" ".join(left)] * dictionary[len(right) - 1][" ".join(right)]
        
    F *= denominator
    return numerator / F

## Dice measure

In [0]:
def dice(words, dictionary):
    words_arr = words.split()
    
    numerator = dictionary[len(words_arr) - 1][words] * 2
    F = 1 / (len(words_arr) - 1)
    denominator = 0
    
    for i in range(len(words_arr) - 1):
        left = words_arr[0:i+1]
        right = words_arr[i+1:]
        
        denominator += dictionary[len(left) - 1][" ".join(left)] + dictionary[len(right) - 1][" ".join(right)]
        
    F *= denominator
    return numerator / F

## LocalMax algorithm

In [0]:
def LocalMax(text, table):
    xys = dict()
    
    for nr in tqdm(range(len(text))):
#     for nr in range(len(text)):
        line = text[nr]

        text_arr = line.split()

        if len(text_arr) < 2:
            continue

        for i in range(len(text_arr) - 2):
            cur_gram = " ".join(text_arr[i:i+2])

            j = i + 3
            n_gram = 1

            while j < len(text_arr) and n_gram < 6:
                next_gram = " ".join(text_arr[i:j])

                try:
                    xys[cur_gram][1] = max(xys[cur_gram][1], table[next_gram])
                    
                except KeyError:
                    if len(cur_gram.split()) > 2:
                        x = table[" ".join(cur_gram.split()[:-1])]
                        x = max(x, table[" ".join(cur_gram.split()[1:])])
                        xys[cur_gram] = [x, table[next_gram]]
                    else:
                        xys[cur_gram] = [-1, table[next_gram]]

                cur_gram = next_gram

                j += 1
                n_gram += 1

            cur_gram = " ".join(text_arr[i:i+2])
            j = i - 1
            n_gram = 1

            while j >= 0 and n_gram < 6:
                next_gram = " ".join(text_arr[j:i+2])


                try:                        
                    xys[cur_gram][1] = max(xys[cur_gram][1], table[next_gram])
                    
                except KeyError:
                    if len(cur_gram.split()) > 2:
                        x = table[" ".join(cur_gram.split()[:-1])]
                        x = max(x, table[" ".join(cur_gram.split()[1:])])
                        xys[cur_gram] = [x, table[next_gram]]
                    else:
                        xys[cur_gram] = [-1, table[next_gram]]


                cur_gram = next_gram

                j -= 1
                n_gram += 1
              
    return xys

## Main function  
In this function the text passed as an argument of the function is pre-processed first. Than the dictionary with n-grams and frequencies is build. Next the desired glue value of each n-gram is calculated, in order to be able to apply LocalMax algorithm and find Relevant Expressions.

In [0]:
def main(text, max_gram, threshold, glue_fun, verbose=False):
    if verbose:
        print("Preprocessing text")
    text = preprocessing(text)
    
    if verbose:
        print("Building dictionary with frequencies")

    dictionary = build_dict(text, max_gram)
    
    table = dict()
    
    if verbose:
        print("Proceeding to calculating glue value")
        print("Glue value calculated for:")

    for i in range(2, max_gram + 1):
        if verbose:
            print(i, "grams")
            
        for s in dictionary[i - 1]:
          table[s] = glue_fun(s, dictionary)
    
    if verbose:
        print("Proceeding to Localmax")
    
    RE = []
    xys = LocalMax(text, table)
            
    for g in xys:
        if xys[g][0] == -1:
            xys[g][0] = xys[g][1]
        
        val = (xys[g][0] + xys[g][1]) / 2
        
        if table[g] > val and re.match(r"^[a-z\'A-Z\s]*$", g) and dictionary[len(g.split()) - 1][g] > threshold:
            RE.append(g)
    
    return RE

## Result of our method for the first paragraph of the text

In [0]:
test = text[0]

RE = main([test], 7, 1, SCP_f)
print("Obtained", len(RE), "Relevant Expressions")
RE

/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm_notebook.py:88: TqdmExperimentalWarning: Detect Google Colab 0.0.1a2 and thus load dummy ipywidgets package. Note that UI is different from that in Jupyter. See https://github.com/tqdm/tqdm/pull/640
  " See https://github.com/tqdm/tqdm/pull/640".format(colab.__version__), TqdmExperimentalWarning)



Obtained 2 Relevant Expressions


['in the', 'John Philoponus']

## Results for the whole corpus  
As we can see, for the corpus of 2 mln words size, using SCP_f function as glue measure, having n-grams of the maximum size of 7 words and threshold for the minimum frequency of Relevant Expression equal 1, we obtain 19059 Relevant Expressions.

In [0]:
RE = main(text, 7, 1, SCP_f, True)
print("Obtained", len(RE), "Relevant Expressions")
results = RE

Preprocessing text
Building dictionary with frequencies
Proceeding to calculating glue value
Glue value calculated for:
2 grams
3 grams
4 grams
5 grams
6 grams
7 grams
Proceeding to Localmax



Obtained 19059 Relevant Expressions


In [0]:
RE[:100]

['crucial role',
 'in the',
 'of the',
 'The first',
 'John Philoponus',
 'on the',
 'and other',
 'communications equipment',
 'based on',
 'with the',
 'divided into',
 'the use of',
 'the study of',
 'known as',
 'what is now',
 'from the',
 'to the',
 'as a',
 'was not',
 'end of',
 'by the',
 'of their',
 'After independence',
 'civil war',
 'such as',
 'the dawn of',
 'the dawn of the',
 'Ottoman Empire',
 'the geopolitical',
 'He also',
 'for the',
 'western Europe',
 'as well as',
 'in the form of',
 'is a',
 'Indic scripts',
 'Southeast Asia',
 'they are',
 'Northern India',
 'Sri Lanka',
 'the site of',
 'Charles VI of France',
 'VI of France',
 'as the',
 'According to',
 'that they',
 'they were',
 'the start of',
 'start of the battle',
 'to be',
 'formula for calculating',
 'is not',
 'Western Church',
 'abbots of monasteries',
 'to a',
 'as an',
 'Roman Catholic',
 'Catholic Church',
 'Orthodox Church',
 'are permitted to be',
 'the rank of',
 'there are',
 'are not',
 '

## Possible improvement  
As we can see in the example above we extracted 100 Relevant Expression from the 2 milion corpus text, Although some of them shouldn't be considered as Relevant Expression. For example "in the" or "at the" shouldn't be considered Relevant Expressions since they are very common words that wont give us great information about the meaning of the text.
As a possible improvement of the method, we propose filtering the list of Relevant Expressions, rejecting those that start with a stop word. For that we have a text file named stopwords.txt with very common words that we can exclude from the Relevant Expressions Extrator giving us a more concrete information about the text. 

In [0]:
file = open("/content/drive/My Drive/STUDIA/SEM 8/pad/stopwords.txt", "r")
stop_words = file.read().split('\n')

In [0]:
def begins_with_stopword(expr):
    first_word = expr.split()[0].lower()
    return first_word in stop_words

In [0]:
RE_without_stopwords = list(filter(lambda x: not begins_with_stopword(x), RE))
RE_without_stopwords[:100]

['crucial role',
 'John Philoponus',
 'communications equipment',
 'based on',
 'divided into',
 'known as',
 'end of',
 'civil war',
 'Ottoman Empire',
 'western Europe',
 'Indic scripts',
 'Southeast Asia',
 'Northern India',
 'Sri Lanka',
 'Charles VI of France',
 'VI of France',
 'According to',
 'start of the battle',
 'formula for calculating',
 'Western Church',
 'abbots of monasteries',
 'Roman Catholic',
 'Catholic Church',
 'Orthodox Church',
 'Catherine II',
 'may be',
 'may be given to any',
 'head of a monastery',
 'celibate priest',
 'World War II',
 'respective countries',
 'Olympic Games',
 'Summer Olympics',
 'one of',
 'one of the most',
 'located in',
 'Karaiskakis Stadium',
 'sports and entertainment',
 'UEFA Cup',
 "Cup Winners' Cup",
 "Winners' Cup Final",
 'United States',
 'literary criticism',
 'Baroque period',
 'murder of Naboth',
 'denunciation of',
 'leading figure',
 'found in',
 'associated with',
 'Albertus Magnus',
 'referred to him as',
 'Middle Ages',

As we can see comparing the 100 Relevant expressions extrated with our stopwords.txt file with the Relevant expressions extrated without the file we can observe that the Relevant expressions extracted with the file are much more informative and give us a better knowledge of the text.

##Using Dice Measure
On the contrary, using dice measure (leaving values of the rest of parameters the same) we obtain 38124 Relevant Expressions.

In [0]:
RE = main(text, 7, 1, dice)
print("Obtained", len(RE), "Relevant Expressions")

Preprocessing text
Building dictionary with frequencies
Proceeding to calculating glue value
Glue value calculated for:
2 grams
3 grams
4 grams
5 grams
6 grams
7 grams
Proceeding to Localmax



Obtained 38124 Relevant Expressions


Below, exemplary 100 obtained Relevant Expressions are presented.

In [0]:
RE[:100]

['crucial role',
 'in the',
 'manuscripts of',
 'of the',
 'The first',
 'John Philoponus',
 'out for',
 'on the',
 'and other',
 'other elements',
 'several centuries',
 'communications equipment',
 'based on',
 'The wording',
 'with the',
 'divided into',
 'or gross',
 'is the',
 'of an',
 'body parts',
 'also includes',
 'use of',
 'known as',
 'and also',
 'also in',
 'what is now',
 'from the',
 'to the',
 'lived there',
 'as a',
 'was not',
 'before the end',
 'by the',
 'only by',
 'After independence',
 'civil war',
 'areas such',
 'such as',
 'which have',
 'have included',
 'included the former',
 'dawn of',
 'establishment of the Ottoman',
 'Ottoman Empire',
 'The Ottomans',
 'over most',
 'many times',
 'led by',
 'and later',
 'later established',
 'He also',
 'which they',
 'for the',
 'western Europe',
 'as well',
 'as well as',
 'in the form of',
 'is a',
 'Indic scripts',
 'Southeast Asia',
 'Today they',
 'they are',
 'are used',
 'used in',
 'South Asia',
 'some othe

## Using the extractor in corpus of 4 million words

In [0]:
file = open("/content/drive/My Drive/STUDIA/SEM 8/pad/en3.67m.txt", "r", encoding="utf8")

whole_text = file.read()
text = whole_text.split('\n')

In [0]:
RE = main(text, 7, 1, SCP_f)
print("Obtained", len(RE), "Relevant Expressions")

Preprocessing text
Building dictionary with frequencies
Proceeding to calculating glue value
Glue value calculated for:
2 grams
3 grams
4 grams
5 grams
6 grams
7 grams
Proceeding to Localmax



Obtained 38351 Relevant Expressions


Extracting Relevant Expressions from the corpus of the size 4 mln words lasted approximately 5 minutes and as a result 38351 Relevant Expressions were obtained. Values of parameters: maximal length of n-gram = 7, minimal frequency threshold of RE = 1, glue measure = SCP_f.  
Below, exemplary 100 REs are presented.

In [0]:
RE[:100]

['alcoholic beverage',
 'that the',
 'on the',
 'the sale of',
 'alcoholic beverages',
 'in the',
 'of the',
 'there are',
 'crucial role',
 'The first',
 'John Philoponus',
 'and other',
 'part of',
 'in the northern part of',
 'northern part',
 'where it',
 'as much as',
 'communications equipment',
 'based on',
 'The assumption',
 'the same',
 'with the',
 'with a',
 'with an',
 'an apogee',
 'and a',
 'to the',
 'orbital period',
 'divided into',
 'of an',
 'the use of',
 'the study of',
 'known as',
 'according to',
 'the second',
 'in terms of',
 'what is now',
 'from the',
 'began to',
 'as a',
 'was not',
 'end of',
 'by the',
 'after the',
 'of their',
 'civil war',
 'such as',
 'decreasing order',
 'most common',
 'for the',
 'number of',
 'and the',
 'the dawn of',
 'the establishment of',
 'Ottoman Empire',
 'Southeast Europe',
 'most of',
 'led by',
 'He also',
 'which they',
 'as well as',
 'in the form of',
 'his father',
 'In the same year',
 'same year',
 'at the',
 'A

## Evaluating the results of the extractor through the Precision, Recall and F-metric

### Precision

In [0]:
from random import randint

count = 0

for i in range(100):
    print(results[randint(0, len(results))])
    verdict = input()
    
    if verdict == 'y':
        count += 1

Logitech Media Server
y
name of several inhabited
n
Caspian Sea
y
nausea and vomiting
y
Jamia Nizamia
y
Daniel Akaka
y
westbound platform
y
largely inhabited
n
Vocational Education
y
paternal grandfather
y
entire field
n
first TV spot
n
States and Europe
n
South Africa
y
drug pellets
y
The movie ends
n
graduated from high school
y
Governor General of Canada
y
court to order
n
another Christian church
n
marriage is to be dissolved
n
Its primary mission
n
sopra Minerva
y
Queensland Heritage Register
y
charting singles
y
pots and pans
y
He died in Paris
n
Dean of the College
y
Tortricidae family
y
Judicial Commissioners
y
top seller
y
Lala Lajpat
y
California legislature
y
capacitive sensing
y
playing professionally
n
involved some significant events
n
convexity assumptions
y
converted to a torpedo
y
Asia Pacific
y
Washington DC
y
Peter Debruge
y
Freiherr of Weissenau
y
broad categories
y
La Plata
y
she also starred
n
Black Bolt
y
Upper West Side
y
since been demolished
n
protein isoforms

In [0]:
FP = 100 - count
TP = count
precision = TP / (FP + TP)
print('Precision: {} %'.format(100 * precision))

Precision: 69.0 %


Taking into consideration 100 randomly chosen Relevant Expressions from the ones extracted by our algorithm from the whole 2 mln words corpus, we can say that 31 of them were incorrectly classified as REs and as a result our method reached 69% of precision. In our opinion, for example, 'few months' or 'Hurler of' cannot be considered as REs.

In [0]:
count = 0

for i in range(100):
    print(RE_without_stopwords[randint(0, len(RE_without_stopwords))])
    verdict = input()
    
    if verdict == 'y':
        count += 1

starring role
y
Album of the Year
y
trying to steal
n
Conservative and Unionist
y
Faustian ideal
y
soundtrack for the film was released
y
Robert Cray
y
Detroit Pistons
y
Marshgate Lane
y
session held on Saturday
n
Distinguished Flying Cross
y
earned run average
n
aims to build
n
Cape Canaveral
y
prize for Category
y
Sufi saint
y
many injuries
n
make sense
y
Narragansett Steamship
y
extremely close
n
eastern Europe
y
commuted into the municipality
y
Days' Battles
y
Cuthbert served in the Georgia
n
Malibu Comics
y
departing Pearl
n
west coast of Graham Land
y
digital downloads
y
abstract concepts
y
Je Khenpo
y
tremendous amount
y
illusory palinopsia
y
trains per hour
n
southern terminus
y
already been in use
n
released two albums
n
Jerry Lawler
y
Copa Libertadores
y
court to order
n
Light Horse Regiment
y
distribution center
y
debut for Yeovil
y
Russian armies
y
forested land
y
fifth round
n
eWorld service
y
Cuthbert served
n
Associated Newspapers
y
Kerman Province
y
Maxwell Evarts
y
Int

In [0]:
FP = 100 - count
TP = count
precision = TP / (FP + TP)
print('Precision: {} %'.format(100 * precision))

Precision: 77.0 %


Taking into consideration 100 randomly chosen Relevant Expressions from the ones extracted by our algorithm from the whole 2 mln words corpus and rejecting those starting with a stopword, we can say that 23 of them were incorrectly classified as REs and as a result our method reached 77% of precision. This small adjustment indeed improved obtainted result.

### Recall

To be able to check the recall measure value for our algorithm we attached to the 2 mln corpus the article from wikipedia website ([link](https://en.wikipedia.org/wiki/CD_Projekt)) which topic is known for us and we can better judge whether an extracted Relevant Expression is indeed relevant.

In [0]:
file = open("/content/drive/My Drive/STUDIA/SEM 8/pad/en1.8mMY.txt", "r", encoding="utf8")

whole_text = file.read()
text = whole_text.split('\n')

Below, two first paragraphs of the article on which we will evaluate recall measure are presented.

In [0]:
import pprint

pp = pprint.PrettyPrinter(indent=4)
test = ''.join(text[-54:-51])
pp.pprint(test)

('CD Projekt S.A. (Polish: [ˌt͡sɛˈdɛ ˈprɔjɛkt]) is a Polish video game '
 'developer, publisher and distributor based in Warsaw, founded in May 1994 by '
 'Marcin Iwiński and Michał Kiciński. Iwiński and Kiciński were video game '
 'retailers before they founded the company, which initially acted as a '
 'distributor of foreign video games for the domestic market. The department '
 'responsible for developing original games, CD Projekt Red, best known for '
 'The Witcher series, was formed in 2002. In 2008, CD Projekt launched the '
 'digital distribution service GOG.com (originally as Good Old Games).The '
 'company began by translating major Western video-game releases into Polish, '
 "collaborating with Interplay Entertainment for two Baldur's Gate games. CD "
 "Projekt was working on the PC version of Baldur's Gate: Dark Alliance when "
 'Interplay experienced financial difficulties. The game was cancelled and the '
 'company decided to reuse the code for their own video game. It b

In [0]:
RE = main(text, 7, 1, SCP_f, True)
print("Obtained", len(RE), "Relevant Expressions")

Preprocessing text
Building dictionary with frequencies
Proceeding to calculating glue value
Glue value calculated for:
2 grams
3 grams
4 grams
5 grams
6 grams
7 grams
Proceeding to Localmax


/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm_notebook.py:88: TqdmExperimentalWarning: Detect Google Colab 0.0.1a2 and thus load dummy ipywidgets package. Note that UI is different from that in Jupyter. See https://github.com/tqdm/tqdm/pull/640
  " See https://github.com/tqdm/tqdm/pull/640".format(colab.__version__), TqdmExperimentalWarning)



Obtained 19089 Relevant Expressions


In [0]:
RE[-30:]

['momentary afterimages',
 'Floriano de Azevedo',
 'Floriano de Azevedo Marques Neto',
 'Azevedo Marques Neto',
 'Jammu Tawi',
 'Rinse FM',
 'Table tennis',
 'Edith Cavell',
 'Jadavpur University',
 "Ernest Hemingway's",
 'scientific payload',
 'short supply',
 'Summer League team',
 'Throughout gameplay',
 'and Teacher of',
 'CD Projekt',
 'Witcher series',
 'Good Old Games',
 'Interplay Entertainment',
 "Baldur's Gate",
 'Interplay experienced financial',
 'console port',
 'Assassins of Kings',
 'help players find',
 "Dark Alliance's cancellation",
 "Alliance's cancellation",
 "Projekt Red's",
 "CD Projekt's",
 'decided to focus on',
 'iOS and Android']

Above the Relevant Expressions extracted from the article are presented.

In [0]:
FN = 3 # digital distribution service, dark alliance, andrzej sapkowski
TP = 5
recall = TP / (FN + TP)
print('Recall: {} %'.format(100 * recall))

Recall: 62.5 %


The value of recall measure was calculated based on our judgement about Relevant Expressions contained by the first 2 paragraphs of the article attached to the corpus. The achieved result is quite satisfying (62.5%). In our opinion, for example 'Dark Alliance' or 'Andrzej Sapkowski' should be cosidered as Relevant Expressions instead of "Dark Alliance's cancellation" or "Alliance's cancellation".



###  F-metric

In [0]:
F1 = 2 * (precision * recall) / (precision + recall)
print('F-metric: {} %'.format(100 * round(F1, 3)))

F-metric: 69.0 %


As we can see, the value of F-metric is equal 69%.